In [2]:
import pandas as pd
import os
from collections import defaultdict
import numpy as np
import re

In [3]:
def def_value(): 
    return 0


def calc_interval(bottom, top):
    
    notes = {
        "C":0,
        "D":1,
        "E":2,
        "F":3,
        "G":4,
        "A":5,
        "B":6
    }

    clockVals = {
        "C": 0,
        "D": 2,
        "E": 4,
        "F": 5,
        "G": 7,
        "A": 9,
        "B": 11
    }

    intervalToClock = [0, 2, 4, 5, 7, 9, 11]

    accidentals = defaultdict(def_value)
    accidentals["--"] = -2
    accidentals["-"] = -1
    accidentals["#"] =  1
    accidentals["##"] = 2    

    bottomNote = re.match("[A-G]", bottom).group()
    topNote = re.match("([A-G])", top).group()
    bAccMatch = re.search("[-#]+", bottom)
    tAccMatch = re.search("[-#]+", top)
    

    bottomAcc = bAccMatch.group() if bAccMatch else " "
    topAcc = tAccMatch.group() if tAccMatch else " "

    bottomInd = notes[bottomNote]
    topInd = notes[topNote]
    noteInterval = (topInd - bottomInd) % 7 + 1

    bottomClock = clockVals[bottomNote] + accidentals[bottomAcc];
    topClock = clockVals[topNote] + accidentals[topAcc];
    clockValue = topClock - bottomClock;

    if clockValue < 0:
        clockValue += 12

    noteToClock = intervalToClock[noteInterval - 1];
    diff = clockValue - noteToClock;

    #The most we'd expect to have to alter an interval is by 6 semitones;
    # otherwise, something is wrong.
    if abs(diff) > 6:
        if diff > 0:
            diff -= 12
        else:
            diff += 12

    ss = "";
    if diff > 0:
        ss = "#"*diff
    elif diff < 0:
        ss = "b" * (-1*diff)
        
    return ss + str(noteInterval)

In [4]:
calc_interval("C##", "G--")

'bbbb5'

In [10]:
os.chdir("../Humdrum/CompleteTranscriptions/")

In [11]:
# os.chdir("../Humdrum/CompleteTranscriptions/Originals/")
files = os.listdir()

In [12]:
files[0].head(10)

NameError: name 'file0' is not defined

In [363]:
files = os.listdir()

In [49]:
files

['Chicago_25Or6To4_1970.hum',
 'KoolAndTheGang_JungleBoogie_1974.hum',
 'MarvinGaye_MercyMercyMe_1971.hum',
 'Commodores_Lady_1981.hum',
 'BruceSpringsteen_ImGoinDown_1985.hum',
 'WilsonPhillips_HoldOn_1990.hum',
 'BingCrosby_SilentNight_1960.hum',
 'Level42_SomethingAboutYou_1986.hum',
 'JerryJeffWalker_MrBojangles_1968.hum',
 'NakedEyes_AlwaysSomethingThereToRemindMe_1983.hum',
 'PeterGabriel_ShockTheMonkey_1982.hum',
 'TheBeatles_LoveMeDo_1964.hum',
 'LedZeppelin_DyerMaker_1973.hum',
 'TheDoors_RidersOnTheStorm_1971.hum',
 'TheKinks_BetterThings_1981.hum',
 'BillyIdol_RebelYell_1984.hum',
 'TomJones_ShesALady_1971.hum',
 'LedZeppelin_OverTheHillsAndFarAway_1973.hum',
 'U2_WithOrWithoutYou_1987.hum',
 'TheWeatherGirls_ItsRainingMen_1983.hum',
 'BruceSpringsteen_MyHometown_1985.hum',
 'TheBeatles_ObLaDiObLaDa_1976.hum',
 'Chicago_DoesAnybodyReallyKnowWhatTimeItIs_1970.hum',
 'GloriaGaynor_NeverCanSayGoodbye_1975.hum',
 'The5thDimension_IfICouldReachYou_1972.hum',
 'MichaelJackson_Beat

In [ ]:
os.chdir("../")

In [50]:
os.getcwd()

'/Users/johnmcnamara/Documents/GitHub/CoCoPops-Billboard-RollingStone-Combined/Billboard/Humdrum/CompleteTranscriptions'

In [52]:
def createHarte(humFile):

    df = pd.read_csv(humFile, sep = "\t")
    chordsym = df["**chordsym"].copy()
    harmony = df["**harmony"].copy()
    df["**harte"] = chordsym
    matches = chordsym.str.extractall(r'([A-G][-#]?)').dropna()
    match_indicies = list(set([matches.index[i][0] for i in range(len(matches.index))]))
    match_indicies.sort()
    proto_harte = chordsym.str.extract("([A-G][-#]?:?\w*\(?[^A-G ]*\)?)").dropna()
    for i in match_indicies:
        notes = matches.loc[i][0]
        if len(notes) > 1:
            bass_interval = calc_interval(notes[0], notes[1])
            bass_interval = "" if bass_interval == "1" else "/" + bass_interval
            proto_harte.loc[i][0] =  proto_harte.loc[i][0] + bass_interval

    for i in proto_harte.index[1:]:
        chordsym.loc[i] = proto_harte.loc[i][0]

    df["**harte"] = chordsym
    df.to_csv("../HarmRhythm/"+humFile, sep="\t")
    return df

In [401]:
createHarte("./AnitaBaker_CaughtUpInTheRapture_1987.hum")

,**kern,**silbe,**harmony,**chordsym,**phrase,**timestamp,**leadinstrument,**harte
0,*ICvox,*,*M4/4,*M4/4,*,*,*,*M4/4
1,*D:,*,*D:,*D:,*,*,*,*D:
2,*M4/4,*,*,*,*,*,*,*
3,*clefC3,*,*,*,*,*,*,*
4,1r;,.,1r;,1r,.,0,1r,1r
...,...,...,...,...,...,...,...,...
729,!!!RDF**timestamp: McGill audio timestamp in s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
730,!!!RDT: 2017/01/12/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
731,"!!!SampleActualRank: 85, 56",NaN,NaN,NaN,NaN,NaN,NaN,NaN
732,"!!!SampleID#: 647, 709",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
for file in files:
    createHarmRhythm(file)

In [56]:
def createHarmRhythm(humFile):
    df = pd.read_csv(humFile, sep = "\t")
    if "**harmony" not in df:
        return
    harmony = df["**harmony"].copy()
    df["**harmrhythm"] = harmony
    matches = harmony.str.extract(r'(^\d+)').dropna()
    for i in matches.index:
        df["**harmrhythm"][i] = matches[0].loc[i]
    df.to_csv("../HarmRhythm/"+humFile, sep="\t")

In [46]:
matches[0].loc[6]

'1'

In [47]:
for i in matches.index:
    df["**harmrhythm"][i] = matches[0].loc[i]

In [48]:
for x in df["**harmrhythm"]:
    print(x)

*M4/4
*
*
*a:
*
*
1
=1
*>Letter>C
*>Label>Intro
1
=2
1
=3
1
=4
2
2
=5
1
=6
1
=7
1
=8
2
2
=9
1
=10
1
=11
1
=12
2
2
=13
1
=14
1
=15
1
=16
2
2
=17
*>Letter>A
*>Label>Verse
1
.
.
.
=18
1
.
.
.
.
=19
1
.
.
=20
2
2
=21
1
.
.
.
=22
1
.
.
.
.
=23
1
.
.
.
=24
2
2
=25
1
.
.
.
=26
1
.
.
.
.
=27
1
.
=28
2
2
=29
1
.
.
.
=30
1
.
.
.
.
=31
1
.
.
.
.
=32
2
2
=33
*>Letter>B
*>Label>Chorus
1
.
.
.
=34
1
.
.
.
.
=35
1
=36
1
.
=37
1
.
.
.
=38
1
.
.
.
.
=39
1
=40
1
.
=41
*>Letter>K
*>Label>Interlude
1
=42
1
=43
1
=44
2
2
=45
1
=46
1
=47
1
=48
2
2
=49
*>Letter>A
*>Label>Verse
1
.
.
.
=50
1
.
.
.
.
=51
1
.
.
=52
2
2
=53
1
.
.
.
=54
1
.
.
.
.
=55
1
.
=56
2
2
=57
1
.
.
.
=58
1
.
.
.
.
=59
1
.
=60
2
2
=61
1
.
.
.
=62
1
.
.
.
.
=63
1
.
.
.
.
=64
2
2
=65
*>Letter>B
*>Label>Chorus
1
.
.
.
=66
1
.
.
.
.
=67
1
=68
1
.
=69
1
.
.
.
=70
1
.
.
.
.
=71
1
.
=72
1
=73
*>Letter>D
*>Label>Instrumental
1
.
.
.
=74
1
=75
1
=76
2
2
=77
1
=78
1
=79
1
=80
2
2
=81
1
=82
1
=83
1
=84
2
2
=85
1
=86
1
=87
1
=88
2
2
=89
1
=90
1
=91
1
=